# 0. Prepare

In [1]:
import zipfile
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

from zipfile import ZipFile

In [2]:
url = '/Users/thienhtt20/Documents/GitHub/Handling-Data/MyData/2019-Nov.csv.zip'

In [3]:
# df = pd.read_csv(url, compression='zip', header=0, sep=',', quotechar='"')
# df.info(verbose=False, memory_usage="deep")

In [4]:
df = pd.read_csv(url, compression='zip', header=0, sep=',', quotechar='"', nrows=10)

In [5]:
df.columns.tolist()

['event_time',
 'event_type',
 'product_id',
 'category_id',
 'category_code',
 'brand',
 'price',
 'user_id',
 'user_session']

# 1.Better read file

### 1.1.Basic Feature Selection For Analysis

1. As we can see, there are 5 features we actually need to do care about:

    [**'category_code', 'brand', 'price', 'event_time', 'event_type'** ]
2. The rest would be outliers and redundant due to its data format and unreadble

3. Data size is enormous ~60M records --> cannot read immediately
4. Data type unoptimized (**float64 -> float16**)

In [6]:
df.head(2)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283


In [7]:
needed_categories = ['category_code', 'brand', 'price', 'event_time', 'event_type']

### 1.2 Optimized Data Type

#### 1.2.1 FLOAT TYPE

In [8]:
df[needed_categories].dtypes

category_code     object
brand             object
price            float64
event_time        object
event_type        object
dtype: object

In [9]:
df = df[needed_categories]

In [10]:
df['price'] = df['price'].astype('float16')

#### 1.2.2 TIME TYPE

In [11]:
df.head(4)

,category_code,brand,price,event_time,event_type
0,electronics.smartphone,xiaomi,489.0000,2019-11-01 00:00:00 UTC,view
1,appliances.sewing_machine,janome,293.7500,2019-11-01 00:00:00 UTC,view
2,NaN,creed,28.3125,2019-11-01 00:00:01 UTC,view
3,appliances.kitchen.washer,lg,713.0000,2019-11-01 00:00:01 UTC,view


In [13]:
df['event_time'] = df['event_time'].apply(lambda x: x.split(' ')[0])

In [16]:
df['event_time'] = pd.to_datetime(df['event_time'])

In [17]:
df['event_time'].dtypes

dtype('<M8[ns]')

In [18]:
df['event_time']

0   2019-11-01
1   2019-11-01
2   2019-11-01
3   2019-11-01
4   2019-11-01
5   2019-11-01
6   2019-11-01
7   2019-11-01
8   2019-11-01
9   2019-11-01
Name: event_time, dtype: datetime64[ns]